# Finite Volume Discretisation

- Finite Volumes, LeVeque book
- Gradient
- Divergence
- Simplify to Laplacian
- Integral
- 

We first import `pybamm` and some useful other modules, and change our working directory to the root of the `PyBaMM` folder:

In [9]:
import pybamm
import numpy as np
import os
import matplotlib.pyplot as plt
from pprint import pprint
os.chdir(pybamm.__path__[0]+'/..')

## Setting up a Discretisation

To set up a discretisation, we must create a geometry, mesh this geometry, and then create the discretisation with the appropriate spatial method(s). The easiest way to create a geometry is to use an inbuilt one:

In [14]:
parameter_values = pybamm.ParameterValues(
    base_parameters={
        "Negative electrode width": 0.3,
        "Separator width": 0.2,
        "Positive electrode width": 0.3,
    }
)

geometry = pybamm.Geometry("1D macro", "1+1D micro")
parameter_values.process_geometry(geometry)

We then use this geometry to create a mesh, which for this example consists of uniform 1D submeshes

In [12]:
submesh_types = {
    "negative electrode": pybamm.Uniform1DSubMesh,
    "separator": pybamm.Uniform1DSubMesh,
    "positive electrode": pybamm.Uniform1DSubMesh,
    "negative particle": pybamm.Uniform1DSubMesh,
    "positive particle": pybamm.Uniform1DSubMesh,
}

var = pybamm.standard_spatial_vars
var_pts = {var.x_n: 40, var.x_s: 25, var.x_p: 35, var.r_n: 10, var.r_p: 10}
mesh = pybamm.Mesh(geometry, submesh_types, var_pts)

Finally, we can use the mesh to create a discretisation, using Finite Volumes as the spatial method for this example

In [13]:
spatial_methods = {
    "macroscale": pybamm.FiniteVolume,
    "negative particle": pybamm.FiniteVolume,
    "positive particle": pybamm.FiniteVolume,
}
disc = pybamm.Discretisation(mesh, spatial_methods)

## Discretisations and Spatial Methods

In this notebook, we will work with three variables `u`, `v`, `w`.

In [26]:
u = pybamm.Variable("u") # u is a variable without a domain (e.g. time, average concentration)
macroscale = ["negative electrode", "separator", "positive electrode"]
v = pybamm.Variable("v", domain=macroscale) # v is a variable in the macroscale (e.g. electrolyte potential)
w = pybamm.Variable("w", domain=["negative particle"]) # w is a variable in the negative particle (e.g. particle concentration)

variables = [u,v,w]

For any spatial method, a `pybamm.Variable` gets converted to a `pybamm.StateVector` which, when evaluated, takes the appropriate slice of the input vector `y`. 

In [27]:
# Pass the list of variables to the discretisation to calculate the slices to be used (order matters here!)
disc.set_variable_slices(variables)

# Discretise the variables
u_disc = disc.process_symbol(u)
v_disc = disc.process_symbol(v)
w_disc = disc.process_symbol(w)

# Print the outcome 
print("Discretised u is: {!r}".format(u_disc))
print("Discretised v is: {!r}".format(v_disc))
print("Discretised w is: {!r}".format(w_disc))

Discretised u is: StateVector(0x49ecb3d3c6115711, y[0:1], children=[], domain=[])
Discretised v is: StateVector(-0x34528f7476fd2c8f, y[1:101], children=[], domain=['negative electrode', 'separator', 'positive electrode'])
Discretised w is: StateVector(0x22bf3b1ad766a83a, y[101:501], children=[], domain=['negative particle'])


Comment on this output

In [37]:
# 
y_scalar = np.array([5])

x = mesh.combine_submeshes(*macroscale)[0].nodes
y_macroscale = x**2

y = np.concatenate([y_scalar, y_macroscale])

In [31]:
print(x)

[0.0046875  0.0140625  0.0234375  0.0328125  0.0421875  0.0515625
 0.0609375  0.0703125  0.0796875  0.0890625  0.0984375  0.1078125
 0.1171875  0.1265625  0.1359375  0.1453125  0.1546875  0.1640625
 0.1734375  0.1828125  0.1921875  0.2015625  0.2109375  0.2203125
 0.2296875  0.2390625  0.2484375  0.2578125  0.2671875  0.2765625
 0.2859375  0.2953125  0.3046875  0.3140625  0.3234375  0.3328125
 0.3421875  0.3515625  0.3609375  0.3703125  0.38       0.39
 0.4        0.41       0.42       0.43       0.44       0.45
 0.46       0.47       0.48       0.49       0.5        0.51
 0.52       0.53       0.54       0.55       0.56       0.57
 0.58       0.59       0.6        0.61       0.62       0.63035714
 0.64107143 0.65178571 0.6625     0.67321429 0.68392857 0.69464286
 0.70535714 0.71607143 0.72678571 0.7375     0.74821429 0.75892857
 0.76964286 0.78035714 0.79107143 0.80178571 0.8125     0.82321429
 0.83392857 0.84464286 0.85535714 0.86607143 0.87678571 0.8875
 0.89821429 0.90892857 0.9196

## Finite Volume Operators

### Gradient operator

The gradient operator is converted to a Matrix-StateVector multiplication

In [43]:
grad_v = pybamm.grad(v)
grad_v_disc = disc.process_symbol(grad_v)
grad_v_disc.render()

@
├── Sparse Matrix (99, 100)
└── y[1:101]


In [46]:
grad_v_disc.children[0].entries.toarray()

array([[-106.66666667,  106.66666667,    0.        , ...,    0.        ,
           0.        ,    0.        ],
       [   0.        , -106.66666667,  106.66666667, ...,    0.        ,
           0.        ,    0.        ],
       [   0.        ,    0.        , -106.66666667, ...,    0.        ,
           0.        ,    0.        ],
       ...,
       [   0.        ,    0.        ,    0.        , ...,   93.33333333,
           0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        , ...,  -93.33333333,
          93.33333333,    0.        ],
       [   0.        ,    0.        ,    0.        , ...,    0.        ,
         -93.33333333,   93.33333333]])

In [44]:
grad_v_disc.evaluate(1, y)

array([0.01875   , 0.0375    , 0.05625   , 0.075     , 0.09375   ,
       0.1125    , 0.13125   , 0.15      , 0.16875   , 0.1875    ,
       0.20625   , 0.225     , 0.24375   , 0.2625    , 0.28125   ,
       0.3       , 0.31875   , 0.3375    , 0.35625   , 0.375     ,
       0.39375   , 0.4125    , 0.43125   , 0.45      , 0.46875   ,
       0.4875    , 0.50625   , 0.525     , 0.54375   , 0.5625    ,
       0.58125   , 0.6       , 0.61875   , 0.6375    , 0.65625   ,
       0.675     , 0.69375   , 0.7125    , 0.73125   , 0.7503125 ,
       0.77      , 0.79      , 0.81      , 0.83      , 0.85      ,
       0.87      , 0.89      , 0.91      , 0.93      , 0.95      ,
       0.97      , 0.99      , 1.01      , 1.03      , 1.05      ,
       1.07      , 1.09      , 1.11      , 1.13      , 1.15      ,
       1.17      , 1.19      , 1.21      , 1.23      , 1.25035714,
       1.27142857, 1.29285714, 1.31428571, 1.33571429, 1.35714286,
       1.37857143, 1.4       , 1.42142857, 1.44285714, 1.46428